# TODO: 概要

# ライブラリのインポート

In [1]:
import os
import json
import asyncio
import datetime

from dotenv import load_dotenv, find_dotenv

from IPython.display import Image, display

from azure.identity.aio import DefaultAzureCredential
from azure.ai.agents.models import (
    FileInfo, FileSearchTool, VectorStore,
    CodeInterpreterTool, FilePurpose,
    ListSortOrder
)

from semantic_kernel.agents import (
    ChatCompletionAgent, ChatHistoryAgentThread,
    AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.contents import (
    ChatMessageContent, FunctionCallContent, FunctionResultContent, AuthorRole, TextContent
)


In [2]:
from semantic_kernel.agents import Agent, ChatCompletionAgent, HandoffOrchestration, OrchestrationHandoffs
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

# 環境変数の取得

In [21]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")
FOUNDRY_FILE_SEARCH_AGENT_ID=os.getenv("FOUNDRY_FILE_SEARCH_AGENT_ID")

# プラグインの作成

In [ ]:
# MCPのエンドポイントURL
MCP_URL = "http://127.0.0.1:8000/mcp/"


mcp_plugin = MCPStreamableHttpPlugin(
    name="mcp_plugin",
    url=MCP_URL
)

await mcp_plugin.connect()

In [ ]:
class OrderStatusPlugin:
    @kernel_function
    def check_order_status(self, order_id: str) -> str:
        return f"Order {order_id} is shipped and will arrive in 2-3 days."

class OrderRefundPlugin:
    @kernel_function
    def process_refund(self, order_id: str, reason: str) -> str:
        print(f"Processing refund for order {order_id} due to: {reason}")
        return f"Refund for order {order_id} has been processed successfully."

class OrderReturnPlugin:
    @kernel_function
    def process_return(self, order_id: str, reason: str) -> str:
        print(f"Processing return for order {order_id} due to: {reason}")
        return f"Return for order {order_id} has been processed successfully."

# クライアントの初期化

In [ ]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# エージェントの作成

In [ ]:
support_agent = ChatCompletionAgent(
    name="TriageAgent",
    description="問い合わせ内容を振り分けるカスタマーサポート担当者です。",
    instructions="お客様からのご要望やお問い合わせに対応してください。",
    service=azure_completion_service,
)

refund_agent = ChatCompletionAgent(
    name="RefundAgent",
    description="返金対応を担当するカスタマーサポート担当者です。",
    instructions="返金に関するご要望に対応してください。",
    service=azure_completion_service,
    plugins=[OrderRefundPlugin()],
)

order_status_agent = ChatCompletionAgent(
    name="OrderStatusAgent",
    description="注文状況の確認を担当するカスタマーサポート担当者です。",
    instructions="注文状況の確認に関するご要望に対応してください。",
    service=azure_completion_service,
    plugins=[OrderStatusPlugin()],
)

order_return_agent = ChatCompletionAgent(
    name="OrderReturnAgent",
    description="返品対応を担当するカスタマーサポート担当者です。",
    instructions="返品に関するご要望に対応してください。",
    service=azure_completion_service,
    plugins=[OrderReturnPlugin()],
)



# オーケストレーションの作成

In [ ]:
# エージェントのレスポンスを処理するコールバック関数
def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"{message.name}: {message.content}")

# ユーザーからの入力を取得する関数
def human_response_function() -> ChatMessageContent:
    user_input = input("User: ")
    print(f"User: {user_input}")
    return ChatMessageContent(role=AuthorRole.USER, content=user_input)

# ハンズオフの定義
handoffs = (
    OrchestrationHandoffs()
    .add_many(
        source_agent=support_agent.name,
        target_agents={
            refund_agent.name: "「返金関連」の問題が発生した場合は、このエージェントにハンズオフしてください",
            order_status_agent.name: " 「注文ステータス」の問題が発生した場合は、このエージェントにハンズオフしてください",
            order_return_agent.name: "「返品関連」の問題が発生した場合は、このエージェントにハンズオフしてください",
            
        },
    )
    .add(
        source_agent=refund_agent.name,
        target_agent=support_agent.name,
        description="「返金関連」の問題ではない場合、このエージェントにハンズオフしてください",
    )
    .add(
        source_agent=order_status_agent.name,
        target_agent=support_agent.name,
        description="「注文ステータス」の問題ではない場合、このエージェントにハンズオフしてください",
    )
    .add(
        source_agent=order_return_agent.name,
        target_agent=support_agent.name,
        description="「返品関連」の問題ではない場合、このエージェントにハンズオフしてください",
    )
)

# ハンドオフのオーケストレーションを実行する関数
async def run_handoff():
    handoff_orchestration = HandoffOrchestration(
        members=[
            support_agent, 
            refund_agent, 
            order_status_agent, 
            order_return_agent
        ],
        handoffs=handoffs,
        agent_response_callback=agent_response_callback,
        human_response_function=human_response_function,
    )
    runtime = InProcessRuntime()
    runtime.start()

    orchestration_result = await handoff_orchestration.invoke(
        task="お客様から電話です。",
        runtime=runtime,
    )

    value = await orchestration_result.get()
    print(value)

    await runtime.stop_when_idle()



In [9]:
await run_handoff()

TriageAgent: ご連絡いただきありがとうございます。お客様からの電話について、どのようなご用件でしょうか？内容（例：注文に関する質問、返品・返金のご希望など）をお知らせいただけますか？ご案内できるようサポートいたします。
User: 発送状況について
TriageAgent: 
TriageAgent: 
TriageAgent: 
OrderStatusAgent: かしこまりました。  
ご注文の発送状況を確認いたします。

お手数ですが、お客様の「注文番号」を教えていただけますか？  
注文番号がわかれば、すぐに発送状況をお調べいたします。
User: 123
OrderStatusAgent: 
OrderStatusAgent: 
OrderStatusAgent: ご注文番号「123」は、すでに発送済みです。  
商品は2～3日以内にお届け予定ですので、到着まで今しばらくお待ちください。

他にご不明な点やご質問がございましたらお知らせください。
User: ありがとうございます。345を返品したいです。
OrderStatusAgent: 
OrderStatusAgent: 
OrderStatusAgent: 
TriageAgent: 
TriageAgent: 
TriageAgent: 
OrderReturnAgent: 返品のご希望を承りました。ご注文番号「345」の返品手続きを進めます。

恐れ入りますが、返品理由をお伺いしてもよろしいでしょうか？  
（例：商品不良、注文間違い、イメージと違った等）

理由を教えていただければ、速やかに対応いたします。
User: 商品不良
Processing return for order 345 due to: 商品不良
OrderReturnAgent: 
OrderReturnAgent: 
OrderReturnAgent: ご注文番号「345」の返品を「商品不良」との理由で手続きいたしました。

返品手続きは完了しております。今後の流れなどご不明な点がありましたらお気軽にお知らせください。

本日はご連絡ありがとうございました。
User: 他に何ができますか？
OrderReturnAgent: お問い合わせありがとうございます。私（オペレーター）では、主に下記のようなご対

# テスト：AzureAIAgent

In [11]:
# AzureAIAgent クライアントを初期化
project_client = AzureAIAgent.create_client(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

In [12]:
code_interpreter_agent_definition = await project_client.agents.get_agent(agent_id=FOUNDRY_CODE_INTERPRETER_AGENT_ID)

In [22]:
file_search_agent_definition = await project_client.agents.get_agent(agent_id=FOUNDRY_FILE_SEARCH_AGENT_ID)

In [13]:
code_interpreter_agent = AzureAIAgent(
    client=project_client,
    definition=code_interpreter_agent_definition,
)

In [23]:
file_search_agent = AzureAIAgent(
    client=project_client,
    definition=file_search_agent_definition,
)

In [ ]:
# エージェントのレスポンスを処理するコールバック関数
# def agent_response_callback(message: ChatMessageContent) -> None:
#     print(f"{message.name}: {message.content}")

def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents.

    Please note that this function is called whenever the agent generates a response,
    including the internal processing messages (such as tool calls) that are not visible
    to other agents in the orchestration.
    """
    print(f"{message.name}: {message.content}")
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"Calling '{item.name}' with arguments '{item.arguments}'")
        if isinstance(item, FunctionResultContent):
            print(f"Result from '{item.name}' is '{item.result}'")

# ユーザーからの入力を取得する関数
def human_response_function() -> ChatMessageContent:
    user_input = input("User: ")
    print(f"User: {user_input}")
    return ChatMessageContent(role=AuthorRole.USER, content=user_input)

# ハンズオフの定義
handoffs = (
    OrchestrationHandoffs()
    .add_many(
        source_agent=support_agent.name,
        target_agents={
            refund_agent.name: "「返金関連」の問題が発生した場合は、このエージェントにハンズオフしてください",
            order_status_agent.name: " 「注文ステータス」の問題が発生した場合は、このエージェントにハンズオフしてください",
            order_return_agent.name: "「返品関連」の問題が発生した場合は、このエージェントにハンズオフしてください",
            # code_interpreter_agent.name: "「計算」や「グラフの出力」の問題が発生した場合は、このエージェントにハンズオフしてください",
            file_search_agent_definition.name: "「ファイル検索」の問題が発生した場合は、このエージェントにハンズオフしてください",
            
        },
    )
    .add(
        source_agent=refund_agent.name,
        target_agent=support_agent.name,
        description="「返金関連」の問題ではない場合、このエージェントにハンズオフしてください",
    )
    .add(
        source_agent=order_status_agent.name,
        target_agent=support_agent.name,
        description="「注文ステータス」の問題ではない場合、このエージェントにハンズオフしてください",
    )
    .add(
        source_agent=order_return_agent.name,
        target_agent=support_agent.name,
        description="「返品関連」の問題ではない場合、このエージェントにハンズオフしてください",
    )
    .add(
        # source_agent=code_interpreter_agent.name,
        # target_agent=support_agent.name,
        # description="「計算」や「グラフの出力」の問題ではない場合、このエージェントにハンズオフしてください",
        source_agent=file_search_agent_definition.name,
        target_agent=support_agent.name,
        description="「ファイル検索」の問題ではない場合、このエージェントにハンズオフしてください",
    )
)

# ハンドオフのオーケストレーションを実行する関数
async def run_handoff():
    handoff_orchestration = HandoffOrchestration(
        members=[
            support_agent, 
            refund_agent, 
            order_status_agent, 
            order_return_agent,
            # code_interpreter_agent
            file_search_agent
        ],
        handoffs=handoffs,
        agent_response_callback=agent_response_callback,
        human_response_function=human_response_function,
    )
    runtime = InProcessRuntime()
    runtime.start()

    orchestration_result = await handoff_orchestration.invoke(
        task="お客様から電話です。",
        runtime=runtime,
    )

    value = await orchestration_result.get()
    print(value)

    await runtime.stop_when_idle()



In [29]:
await run_handoff()

TriageAgent: ご連絡ありがとうございます。どのようなご用件でしょうか？お手伝いできることがございましたらお知らせください。
User: ファイル検索をお願いしたいです。
TriageAgent: 
Calling 'Handoff-transfer_to_service_docs_search_agent' with arguments '{}'
TriageAgent: 
Result from 'Handoff-transfer_to_service_docs_search_agent' is 'None'
TriageAgent: 
service_docs_search_agent: 承知しました。検索したい内容やキーワードを教えていただけますか？  
お客様がお探しの情報について、できるだけ具体的にご指示いただけるとスムーズにご案内できます。
User: PCの出荷日を教えて。
service_docs_search_agent: 
Calling 'file_search' with arguments '{'ranking_options': {'ranker': 'default_2024_08_21', 'score_threshold': 0.0}}'
Result from 'file_search' is '[]'
service_docs_search_agent: PC（パソコン）の出荷日は「受注確認日から7営業日以内」となっております。なお、設置工事が必要な商品やメーカーからの取り寄せが必要な場合は、さらに日数がかかる場合があります。詳細は各商品ページの記載もご確認ください【8:1†出荷日ポリシー.pdf】。
User: ありがとうございます。以前注文した商品の返品対応をお願いしたいです。
service_docs_search_agent: 
Calling 'file_search' with arguments '{'ranking_options': {'ranker': 'default_2024_08_21', 'score_threshold': 0.0}}'
Result from 'file_search' is '[]'
service_docs_search_agent: 返品対応についてご案内いたしま